In [ ]:
import os, shutil

#必要なライブラリのインポート
import re, gc, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import warnings, random
import cv2

warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
import torchvision.models as models
from torch.cuda.amp import GradScaler

import timm
import yaml
from tqdm import tqdm
import time
import copy
from collections import defaultdict

from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

In [ ]:
ARGS = {
  'DATA_DIR': '/root/signate_tecno/input/',
  'OUT_DIR': '/root/signate_tecno/output',
  'model_name': 'vit_l_16',
  'image_size': (224, 224), # vit_l16
  #cpu(slow)
  #'train_batch_size': 4,
  #'test_batch_size': 8,
  #gpu
  #'train_batch_size': 28, # 32(x)
  #'test_batch_size': 56,
  #'n_fold': 2,
  #'epochs': 3,
  #'timm_model_name': 'resnet50',
  #'timm_model_name': 'vit_base_patch16_224',
  #lb:0.8545424 ? :
  #'timm_model_name': 'vit_large_patch32_224_in21k',
  #down?
  #'timm_model_name': 'vit_huge_patch14_224_in21k',
  #colab free crash memory?.(batch=4)
  #'timm_model_name': 'vit_giant_patch14_224_clip_laion2b',
  #Only one class present in y_true. ROC AUC score is not defined in that case.
  #lb:0.8944379  : top, batchsize : 4, 'image_size': (448, 448),
  #'timm_model_name': 'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k',
  'timm_model_name': "timm/vit_mediumd_patch16_reg4_gap_384.sbb2_e200_in12k_ft_in1k"
  'pretrained': True,
  'n_fold': 2, # 5
  'epochs': 2, # 8
  'image_size': (384, 384), # eva02_large_patch14_448
  'criterion': 'CrossEntropy',
  #'is_blurry': True,
  'is_blurry': False,
  #'image_size': (336, 336),
  #GPU: 16GB
  'train_batch_size': 5, # 4, #1(ng?)
  'test_batch_size': 15, #x3?
  #GPU: 80GB?
  #'train_batch_size': 32, # 4, #1(ng?)
  #'test_batch_size': 96,
  #batchsize row is roc_auc calc ng,(batchsizeが小さいと、class data が片方のみ存在している状態になり roc_auc の計算ができない)
  'seed': 2023,
  'optimizer': 'AdamW',
  'learning_rate': 1e-05,
  'scheduler': 'CosineAnnealingLR', # CosineAnnealingWarmRestarts
  'min_lr': 1e-06,
  'T_max': 500,
  'n_accumulate': 1,
  'clip_grad_norm': 'None',
  'apex': True,
  'num_classes': 2,
  'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  }

In [ ]:
def create_folds(data, num_splits, seed):
    data["kfold"] = -1

    mskf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)
    labels = ["label"]
    data_labels = data[labels].values

    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        data.loc[v_, "kfold"] = f

    return data

train = pd.read_csv(f"{ARGS['DATA_DIR']}/train.csv")
train = create_folds(train, num_splits=ARGS["n_fold"], seed=ARGS["seed"])
print("Folds created successfully")

train.head()

In [ ]:
import albumentations as albu
from albumentations.pytorch import transforms as AT

# Augumentation用
#CV : up, LB : down
image_transform_train = albu.Compose([
    albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.RandomBrightnessContrast(p=0.3),
    albu.RandomGamma(gamma_limit=(85, 115), p=0.3),
    albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.10, rotate_limit=45, p=0.5),
    albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AT.ToTensorV2()
])

image_transform = albu.Compose([
    albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
    # albu.HorizontalFlip(p=0.5),
    # albu.VerticalFlip(p=0.5),
    # albu.RandomBrightnessContrast(p=0.3),
    # albu.RandomGamma(gamma_limit=(85, 115), p=0.3),
    # albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.10, rotate_limit=45, p=0.5),
    albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AT.ToTensorV2()
])